# Scikit-learn

In [31]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error

In [16]:
df = pd.read_hdf("train_data.h5")
print(df.shape)
print(df.head())

(22732, 8)
                                           geo_block  \
0       [г. Москва, Лианозово, г. Москва, Лианозово]   
1           [г. Москва, Ховрино, г. Москва, Ховрино]   
2  [г. Москва, ул Лобачевского, г. Москва, ул Лоб...   
3  [г. Москва, Ховрино, ул Дыбенко, г. Москва, Хо...   
4   [г. Москва, Даниловский, г. Москва, Даниловский]   

                                         breadcrumbs         price  \
0    [Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]   5 402 084 ₽   
1         [Москва, Ховрино, м. Ховрино, МЦК Коптево]   6 827 990 ₽   
2        [Москва, МЦК Кутузовская, ул. Лобачевского]   8 680 000 ₽   
3  [Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....   8 624 435 ₽   
4  [Москва, Даниловский, м. Автозаводская, МЦК Ав...  29 500 000 ₽   

                            owner  \
0                              []   
1                              []   
2  [ЖК "Крылья". Доп. скидка 5%!]   
3                              []   
4                              []   

        

In [19]:
def parse_price(x):
    return int(x.split("₽")[0].replace(" ", ""))

df["price_num"] = df["price"].map(parse_price).map(lambda x: x/1000000)
df['price_num']

0          5.402084
1          6.827990
2          8.680000
3          8.624435
4         29.500000
            ...    
22727     12.650000
22728     17.610000
22729     14.941300
22730     21.610000
22731    110.000000
Name: price_num, Length: 22732, dtype: float64

In [20]:
df["params"].head(2).values

array([{'Охрана:': 'закрытая территория', 'Тип здания:': 'Монолитное', 'Тип объекта:': 'квартиры', 'Количество корпусов:': '1', 'Тип объявления:': 'в новостройке', 'Застройщик:': 'Доходный дом', 'Общая площадь:': '54.3 м²', 'Дата публикации:': '22 апреля', 'Количество комнат:': '2', 'Парковка:': 'отдельная многоуровневая, гостевая', 'Дата  обновления:': '18 мая', 'Количество этажей:': '11', 'Сдача:': '1 кв. 2020 года', 'Комиссия агенту:': 'без комиссии', 'Высота потолков:': '2.8 м', 'Этаж:': '8/11', 'Этап строительства:': 'Возведение стен', 'Новостройка:': 'Апарт-комплекс Nord, м. Алтуфьево', 'Количество квартир:': '163', 'Класс жилья:': 'Комфорт класс', 'Адрес:': ''},
       {'Лифт:': 'да', 'Тип здания:': 'Монолитное', 'Тип объекта:': 'квартира', 'Количество корпусов:': '5', 'Тип объявления:': 'в новостройке', 'Застройщик:': 'MR Group', 'Общая площадь:': '38 м²', 'Вид из окна:': 'на улицу', 'Количество комнат:': '2', 'Парковка:': 'подземная', 'Дата  обновления:': '16 мая', 'Класс жиль

### Первая строка (недвижимость)
```
{
    'Охрана:': 'закрытая территория', 
    'Тип здания:': 'Монолитное', 
    'Тип объекта:': 'квартиры', 
    'Количество корпусов:': '1', 
    'Тип объявления:': 'в новостройке', 
    'Застройщик:': 'Доходный дом', 
    'Общая площадь:': '54.3 м²', 
    'Дата публикации:': '22 апреля', 
    'Количество комнат:': '2', 
    'Парковка:': 'отдельная многоуровневая, гостевая', 
    'Дата  обновления:': '18 мая', 
    'Количество этажей:': '11', 
    'Сдача:': '1 кв. 2020 года', 
    'Комиссия агенту:': 'без комиссии', 
    'Высота потолков:': '2.8 м', 
    'Этаж:': '8/11', 
    'Этап строительства:': 
    'Возведение стен', 'Новостройка:': 
    'Апарт-комплекс Nord, м. Алтуфьево', 
    'Количество квартир:': '163', 
    'Класс жилья:': 'Комфорт класс', 
    'Адрес:': ''}
```


### Вторая строка (недвижимость)
```
{
    'Лифт:': 'да', 
    'Тип здания:': 'Монолитное', 
    'Тип объекта:': 'квартира', 
    'Количество корпусов:': '5', 
    'Тип объявления:': 'в новостройке', 
    'Застройщик:': 'MR Group', 
    'Общая площадь:': '38 м²', 
    'Вид из окна:': 'на улицу', 
    'Количество комнат:': '2', 
    'Парковка:': 'подземная', 
    'Дата  обновления:': '16 мая', 
    'Класс жилья:': 'Комфорт класс', 
    'Дата публикации:': '10 мая', 
    'Количество этажей:': '31', 
    'Сдача:': '2 кв. 2021 года', 
    'Комиссия агенту:': 'без комиссии', 
    'Высота потолков:': '3.15 м', 
    'Этаж:': '6/31', 
    'Этап строительства:': 'Возведение стен', 
    'Новостройка:': 'ЖК Discovery (ЖК «Дискавери»), м. Ховрино', 
    'Количество квартир:': '1251', 
    'Возможна ипотека:': 'да', 
    'Адрес:': ''}
```

In [21]:
params = df["params"].apply(pd.Series)
print(params.shape)
params

(22732, 55)


,Охрана:,Тип здания:,Тип объекта:,Количество корпусов:,Тип объявления:,Застройщик:,Общая площадь:,Дата публикации:,Количество комнат:,Парковка:,...,Возможен торг:,Класс:,Телевизор:,Тип фундамента:,Количество жилых помещений:,Детская площадка:,Стиральная машина:,Наибольшее количество этажей:,Наименьшее количество этажей:,Предоплата:
0,закрытая территория,Монолитное,квартиры,1,в новостройке,Доходный дом,54.3 м²,22 апреля,2,"отдельная многоуровневая, гостевая",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Монолитное,квартира,5,в новостройке,MR Group,38 м²,10 мая,2,подземная,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,закрытая территория,Монолитное,квартиры,3,от застройщика,АО «Лидер Инвест»,40 м²,20 марта,1,"подземная на 1103 м/м, открытая",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Монолитное,квартира,5,в новостройке,MR Group,43 м²,29 апреля,2,подземная,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Монолитное,квартиры,NaN,вторичка,NaN,141.1 м²,15 декабря 2018,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22727,NaN,Панельное,квартиры,NaN,вторичка,NaN,55 м²,29 января,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22728,NaN,Монолитное,квартиры,13,от застройщика,АО «Порт-Сити»,75 м²,3 апреля,3,подземная на 305 м/м,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22729,предусмотрена,Монолитно-кирпичное,квартира,15,от застройщика,ГК «МонАрх»,79.9 м²,16 мая,2,"подземная/наземная, 1796 м/м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22730,предусмотрена,Монолитное,квартиры,4,в новостройке,Capital Group,135 м²,24 апреля,2,подземная на 1359 м/м,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
params = params.fillna(-1)
params

,Охрана:,Тип здания:,Тип объекта:,Количество корпусов:,Тип объявления:,Застройщик:,Общая площадь:,Дата публикации:,Количество комнат:,Парковка:,...,Возможен торг:,Класс:,Телевизор:,Тип фундамента:,Количество жилых помещений:,Детская площадка:,Стиральная машина:,Наибольшее количество этажей:,Наименьшее количество этажей:,Предоплата:
0,закрытая территория,Монолитное,квартиры,1,в новостройке,Доходный дом,54.3 м²,22 апреля,2,"отдельная многоуровневая, гостевая",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,Монолитное,квартира,5,в новостройке,MR Group,38 м²,10 мая,2,подземная,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,закрытая территория,Монолитное,квартиры,3,от застройщика,АО «Лидер Инвест»,40 м²,20 марта,1,"подземная на 1103 м/м, открытая",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,Монолитное,квартира,5,в новостройке,MR Group,43 м²,29 апреля,2,подземная,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,Монолитное,квартиры,-1,вторичка,-1,141.1 м²,15 декабря 2018,4,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22727,-1,Панельное,квартиры,-1,вторичка,-1,55 м²,29 января,2,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
22728,-1,Монолитное,квартиры,13,от застройщика,АО «Порт-Сити»,75 м²,3 апреля,3,подземная на 305 м/м,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
22729,предусмотрена,Монолитно-кирпичное,квартира,15,от застройщика,ГК «МонАрх»,79.9 м²,16 мая,2,"подземная/наземная, 1796 м/м",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
22730,предусмотрена,Монолитное,квартиры,4,в новостройке,Capital Group,135 м²,24 апреля,2,подземная на 1359 м/м,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [23]:
obj_feats = params.select_dtypes(object).columns
obj_feats

Index(['Охрана:', 'Тип здания:', 'Тип объекта:', 'Количество корпусов:',
       'Тип объявления:', 'Застройщик:', 'Общая площадь:', 'Дата публикации:',
       'Количество комнат:', 'Парковка:', 'Дата  обновления:',
       'Количество этажей:', 'Сдача:', 'Комиссия агенту:', 'Высота потолков:',
       'Этаж:', 'Этап строительства:', 'Новостройка:', 'Количество квартир:',
       'Класс жилья:', 'Адрес:', 'Лифт:', 'Вид из окна:', 'Возможна ипотека:',
       'Год постройки:', 'Этажность:', 'Мусоропровод:', 'Ремонт:',
       'Площадь кухни:', 'Жилая комната:', 'Тип санузла:', 'Тип балкона:',
       'Мебель на кухне:', 'Холодильник:', 'Интернет:', 'Мебель:', 'Телефон:',
       'Свободная планировка:', 'Управляющая компания:',
       'Количество подъездов:', 'Тип дома:', 'Тип комнат:', 'Покрытие пола:',
       'Серия:', 'Тип перекрытия:', 'Возможен торг:', 'Класс:', 'Телевизор:',
       'Тип фундамента:', 'Количество жилых помещений:', 'Детская площадка:',
       'Стиральная машина:', 'Наиболь

In [24]:
for feat in obj_feats:
    df["{}_cat".format(feat)] = params[feat].factorize()[0]
    

cat_feats = [x for x in df.columns if "_cat" in x]
cat_feats

['Охрана:_cat',
 'Тип здания:_cat',
 'Тип объекта:_cat',
 'Количество корпусов:_cat',
 'Тип объявления:_cat',
 'Застройщик:_cat',
 'Общая площадь:_cat',
 'Дата публикации:_cat',
 'Количество комнат:_cat',
 'Парковка:_cat',
 'Дата  обновления:_cat',
 'Количество этажей:_cat',
 'Сдача:_cat',
 'Комиссия агенту:_cat',
 'Высота потолков:_cat',
 'Этаж:_cat',
 'Этап строительства:_cat',
 'Новостройка:_cat',
 'Количество квартир:_cat',
 'Класс жилья:_cat',
 'Адрес:_cat',
 'Лифт:_cat',
 'Вид из окна:_cat',
 'Возможна ипотека:_cat',
 'Год постройки:_cat',
 'Этажность:_cat',
 'Мусоропровод:_cat',
 'Ремонт:_cat',
 'Площадь кухни:_cat',
 'Жилая комната:_cat',
 'Тип санузла:_cat',
 'Тип балкона:_cat',
 'Мебель на кухне:_cat',
 'Холодильник:_cat',
 'Интернет:_cat',
 'Мебель:_cat',
 'Телефон:_cat',
 'Свободная планировка:_cat',
 'Управляющая компания:_cat',
 'Количество подъездов:_cat',
 'Тип дома:_cat',
 'Тип комнат:_cat',
 'Покрытие пола:_cat',
 'Серия:_cat',
 'Тип перекрытия:_cat',
 'Возможен торг:

In [25]:
X = df[cat_feats]
y = df["price_num"]

X.shape, y.shape

((22732, 55), (22732,))

## Train/Predict the model + Absolute error calculations

In [26]:
model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

In [27]:
mean_absolute_error(y, y_pred)

13.205545368030423

In [28]:
df["price_num"].mean()

17.5255080538888

In [39]:
model = DecisionTreeRegressor(max_depth=7)
model.fit(X, y)
y_pred = model.predict(X)

mean_absolute_error(y, y_pred)

5.98023964766517

### Decision Tree Regressor model has shown less error, so we can use it to predict the price